# MERIT API Tutorial: Complete Developer Guide

This notebook shows you how to use the MERIT API package in your own projects. We'll demonstrate the **before and after** - showing you the complex, error-prone code you'd write without MERIT, and the simple, robust code you can write with MERIT.

## 🎯 What You'll Learn

- How to replace complex API code with simple MERIT clients
- Why MERIT's features matter for production applications
- Real-world patterns you can copy-paste into your projects
- How to build robust, scalable LLM applications

## 📋 Prerequisites

- Python 3.8+
- OpenAI API key (optional, for real examples)
- Gemini API key (optional, for real examples)

## 🚀 Installation

```bash
pip install merit
```

In [ ]:
# API Key Configuration - Replace with your actual keys
# You can also set these as environment variables: OPENAI_API_KEY and GOOGLE_API_KEY

# Option 1: Set your keys directly (not recommended for production)
OPENAI_API_KEY = ""  # Replace with your actual OpenAI API key
GOOGLE_API_KEY = ""   # Replace with your actual Google API key

# Option 2: Load from environment variables (recommended)
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', OPENAI_API_KEY)
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY', GOOGLE_API_KEY)

# For demo purposes, we'll use placeholder keys that will trigger graceful error handling
DEMO_OPENAI_KEY = "sk-demo-key-for-examples"  # This will fail gracefully in examples
DEMO_GOOGLE_KEY = "demo-google-key-for-examples"  # This will fail gracefully in examples

print("🔑 API Keys configured!")
print(f"OpenAI key configured: {'✅' if OPENAI_API_KEY and OPENAI_API_KEY != 'your-openai-api-key-here' else '❌ (using demo key)'}")
print(f"Google key configured: {'✅' if GOOGLE_API_KEY and GOOGLE_API_KEY != 'your-google-api-key-here' else '❌ (using demo key)'}")
print("\n💡 To use real API keys:")
print("   1. Replace the placeholder keys above with your actual keys, OR")
print("   2. Set OPENAI_API_KEY and GOOGLE_API_KEY environment variables")


🔑 API Keys configured!
OpenAI key configured: ✅
Google key configured: ✅

💡 To use real API keys:
   1. Replace the placeholder keys above with your actual keys, OR
   2. Set OPENAI_API_KEY and GOOGLE_API_KEY environment variables


## 1. Getting Started: Your First API Call

Let's start with the most basic example - making an API call to generate text.

### ❌ WITHOUT MERIT: Complex and Error-Prone

Here's what you'd typically write without MERIT:

In [49]:
# This is what most developers write - lots of boilerplate, no error handling
import requests
import os

def generate_text_without_merit(prompt):
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise ValueError('API key not found')
    
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    
    data = {
        'model': 'gpt-3.5-turbo',
        'messages': [{'role': 'user', 'content': prompt}],
        'temperature': 0.7
    }
    
    response = requests.post(
        'https://api.openai.com/v1/chat/completions',
        headers=headers,
        json=data
    )
    
    # No error handling, no retries, no validation
    response.raise_for_status()
    return response.json()['choices'][0]['message']['content']

# Usage - fragile and verbose
try:
    result = generate_text_without_merit('What is artificial intelligence?')
    print(f'Result: {result}')
except Exception as e:
    print(f'Error: {e}')

Result: Artificial intelligence (AI) refers to the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, problem-solving, perception, and language understanding. AI can be used to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.


### ✅ WITH MERIT: Simple and Robust

Here's the same functionality with MERIT:

In [50]:
# Clean, simple, and robust
from merit.api import OpenAIClient

# One line setup - automatically loads from environment, includes error handling
client = OpenAIClient()

# One line usage - includes retries, validation, and proper error handling
result = client.generate_text('What is artificial intelligence?')
print(f'Result: {result}')

Result: Artificial intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, problem-solving, perception, and decision-making. AI can be used to automate tasks, analyze data, recognize patterns, and make predictions. It is a rapidly evolving field with applications in various industries such as healthcare, finance, transportation, and more.


### 🎯 Key Benefits

- **90% less code** - 2 lines vs 20+ lines
- **Automatic error handling** - Built-in retries and validation
- **Environment integration** - Automatically loads API keys
- **Production ready** - Includes logging, monitoring, and best practices

## 2. Multi-Provider Support: Write Once, Use Anywhere

One of MERIT's biggest advantages is that you can switch between different LLM providers without changing your code.

### ❌ WITHOUT MERIT: Separate Code for Each Provider

In [51]:
# Separate implementation for OpenAI
def openai_generate(prompt):
    # 20+ lines of OpenAI-specific code
    import requests
    import os
    headers = {'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}'}
    data = {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'user', 'content': prompt}]}
    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=data)
    return response.json()['choices'][0]['message']['content']

# Completely different implementation for Gemini
def gemini_generate(prompt):
    # 20+ lines of Gemini-specific code
    from google import genai
    client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))
    response = client.models.generate_content(model='gemini-2.0-flash-exp', contents=prompt)
    return response.text

# Your application code needs to know about each provider
def my_app_function(prompt, provider='openai'):
    if provider == 'openai':
        return openai_generate(prompt)
    elif provider == 'gemini':
        return gemini_generate(prompt)
    else:
        raise ValueError(f'Unknown provider: {provider}')

# Switching providers requires code changes
result1 = my_app_function('Hello', 'openai')
result2 = my_app_function('Hello', 'gemini')
print(f'OpenAI: {result1}')
print(f'Gemini: {result2}')

OpenAI: Hello! How can I assist you today?
Gemini: Hello! How can I help you today?



### ✅ WITH MERIT: Polymorphic Usage

With MERIT, the same code works with any provider:

In [52]:
from merit.api import OpenAIClient, GeminiClient

# Both clients implement the same interface
openai_client = OpenAIClient()
gemini_client = GeminiClient()

# Your application code is provider-agnostic
def my_app_function(client, prompt):
    # Same code works with any client!
    return client.generate_text(prompt)

# Switch providers by just changing the client
result1 = my_app_function(openai_client, 'Hello')
result2 = my_app_function(gemini_client, 'Hello')

print(f'OpenAI: {result1}')
print(f'Gemini: {result2}')

# Easy A/B testing
clients = [openai_client, gemini_client]
for i, client in enumerate(clients):
    result = my_app_function(client, 'What is AI?')
    print(f'Client {i+1}: {result[:50]}...')

OpenAI: Hello! How can I assist you today?
Gemini: Hello! How can I help you today?

Client 1: AI, or artificial intelligence, refers to the simu...
Client 2: AI, or Artificial Intelligence, is a broad field o...


## 3. Configuration Management: Production-Ready Setup

Managing API keys, endpoints, and configuration is crucial for production applications.

### ❌ WITHOUT MERIT: Scattered Configuration

Configuration is typically scattered and hard to manage:

In [53]:
# Configuration scattered throughout your code
import os

# Hardcoded values mixed with environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  # What if it's missing?
OPENAI_BASE_URL = 'https://api.openai.com/v1'  # Hardcoded
MODEL = 'gpt-3.5-turbo'  # Hardcoded
TEMPERATURE = 0.7  # Hardcoded

# No validation - fails at runtime
if not OPENAI_API_KEY:
    raise ValueError('Missing API key')

# Configuration repeated in every function
def make_api_call():
    headers = {'Authorization': f'Bearer {OPENAI_API_KEY}'}
    data = {'model': MODEL, 'temperature': TEMPERATURE}
    # ... rest of the code
    pass

# Different configuration for each provider
GEMINI_API_KEY = os.getenv('GOOGLE_API_KEY')
GEMINI_MODEL = 'gemini-2.0-flash-exp'
# More scattered config...

### ✅ WITH MERIT: Centralized Configuration

MERIT provides multiple ways to configure your clients:

In [54]:
from merit.api import OpenAIClient, OpenAIClientConfig, GeminiClient, GeminiClientConfig

# Method 1: Environment variables (recommended for production)
# Just set OPENAI_API_KEY and GOOGLE_API_KEY in your environment
print('=== Method 1: Environment Variables ===')
openai_client = OpenAIClient()  # Automatically loads from environment
gemini_client = GeminiClient()  # Automatically loads from environment

print(f'OpenAI authenticated: {openai_client.is_authenticated}')
print(f'Gemini authenticated: {gemini_client.is_authenticated}')

# Method 2: Direct parameters (good for development)
print('\n=== Method 2: Direct Parameters ===')
openai_direct = OpenAIClient(
    api_key=OPENAI_API_KEY,
    model='gpt-4',
    organization_id='your-org'
)

gemini_direct = GeminiClient(
    api_key=GOOGLE_API_KEY,
    generation_model='gemini-2.0-flash-exp',
    temperature=0.1
)

print(f'OpenAI model: {openai_direct.model}')
print(f'Gemini model: {gemini_direct.generation_model}')

# Method 3: Configuration objects (best for complex setups)
print('\n=== Method 3: Configuration Objects ===')
openai_config = OpenAIClientConfig(
    api_key=OPENAI_API_KEY,
    model='gpt-4',
    embedding_model='text-embedding-ada-002',
    organization_id='your-org'
)

gemini_config = GeminiClientConfig(
    api_key=GOOGLE_API_KEY,
    generation_model='gemini-2.0-flash-exp',
    embedding_model='text-embedding-004',
    temperature=0.1,
    max_output_tokens=1024
)

# Validate configuration before use
openai_config.validate()
gemini_config.validate()

openai_from_config = OpenAIClient(config=openai_config)
gemini_from_config = GeminiClient(config=gemini_config)

print(f'Config validation passed for both clients')
print(f'OpenAI embedding model: {openai_from_config.embedding_model}')
print(f'Gemini temperature: {gemini_from_config.temperature}')

=== Method 1: Environment Variables ===
OpenAI authenticated: True
Gemini authenticated: True

=== Method 2: Direct Parameters ===
OpenAI model: gpt-4
Gemini model: gemini-2.0-flash-exp

=== Method 3: Configuration Objects ===
Config validation passed for both clients
OpenAI embedding model: text-embedding-ada-002
Gemini temperature: 0.1


### 🔄 Easy Model and Parameter Switching

The real power of MERIT's configuration system is how easy it is to switch models and parameters:

In [55]:
# Define different configurations for different use cases

# Configuration 1: Creative writing (high temperature)
creative_config = OpenAIClientConfig(
    api_key=OPENAI_API_KEY,
    model='gpt-4',
    temperature=0.9,
    max_tokens=200
)

# Configuration 2: Factual answers (low temperature)
factual_config = OpenAIClientConfig(
    api_key=OPENAI_API_KEY,
    model='gpt-3.5-turbo',
    temperature=0.1,
    max_tokens=100
)

# Configuration 3: Cost-effective (cheaper model)
budget_config = OpenAIClientConfig(
    api_key=OPENAI_API_KEY,
    model='gpt-3.5-turbo',
    temperature=0.5,
    max_tokens=50
)

# Create clients from configs
creative_client = OpenAIClient(config=creative_config)
factual_client = OpenAIClient(config=factual_config)
budget_client = OpenAIClient(config=budget_config)

# Same prompt, different behaviors
prompt = 'Write about the future of AI'

print('=== Same Prompt, Different Configurations ===')

print('🎨 CREATIVE (GPT-4, temp=0.9):')
creative_response = creative_client.generate_text(prompt)
print(f'{creative_response}\n')

print('📊 FACTUAL (GPT-3.5, temp=0.1):')
factual_response = factual_client.generate_text(prompt)
print(f'{factual_response}\n')

print('💰 BUDGET (GPT-3.5, temp=0.5, 50 tokens):')
budget_response = budget_client.generate_text(prompt)
print(f'{budget_response}\n')

=== Same Prompt, Different Configurations ===
🎨 CREATIVE (GPT-4, temp=0.9):
Artificial Intelligence (AI) is one of the most transformative technologies of our time and its potential to revolutionize various sectors is vast. The future of AI is promising and holds great potential for reshaping our lives, our businesses, and our societies.

In the coming years, AI is expected to make significant advancements. It is projected that AI will become more sophisticated and autonomous, capable of learning and making decisions without human intervention. This increased autonomy will enable AI to perform complex tasks more efficiently than humans. For instance, autonomous vehicles, powered by AI, are expected to become a common sight on our roads, reducing human error and improving road safety.

AI is also expected to advance in terms of natural language processing, image recognition, and predictive analytics. These advancements will enable AI to better understand, interpret, and respond to human

### 🔄 Cross-Provider Configuration Flexibility

The same configuration approach works across all providers:

In [56]:
# Define equivalent configurations for different providers

# OpenAI Configuration for creative writing
openai_creative = OpenAIClientConfig(
    api_key=OPENAI_API_KEY,
    model='gpt-4',
    temperature=0.8,
    max_tokens=150
)

# Gemini Configuration for creative writing
gemini_creative = GeminiClientConfig(
    api_key=GOOGLE_API_KEY,
    generation_model='gemini-2.0-flash-exp',
    temperature=0.8,
    max_output_tokens=150
)

# Create clients from configs
openai_writer = OpenAIClient(config=openai_creative)
gemini_writer = GeminiClient(config=gemini_creative)

# Same application logic works with both
def creative_writing_app(client, topic):
    prompt = f'Write a creative short story about {topic}'
    return client.generate_text(prompt)

# Easy to switch providers
topic = 'time travel'

print('=== Same App Logic, Different Providers ===')

print('📝 OpenAI Creative Story:')
openai_story = creative_writing_app(openai_writer, topic)
print(f'{openai_story[:100]}...\n')

print('📝 Gemini Creative Story:')
gemini_story = creative_writing_app(gemini_writer, topic)
print(f'{gemini_story[:100]}...\n')

# Easy A/B testing across providers
writers = [
    ('OpenAI GPT-4', openai_writer),
    ('Gemini Flash', gemini_writer)
]

print('🔬 A/B Testing Results:')
for name, writer in writers:
    result = creative_writing_app(writer, 'robots')
    print(f'{name}: {len(result)} characters generated')
    print(f'Preview: {result[:80]}...\n')

=== Same App Logic, Different Providers ===
📝 OpenAI Creative Story:
Title: The Hourglass of Eternity

In the bustling city of New York, a timid librarian named Stanley ...

📝 Gemini Creative Story:
Elara traced the glyphs on the weathered brass plate, her fingers ghosting over symbols that felt an...

🔬 A/B Testing Results:
OpenAI GPT-4: 3319 characters generated
Preview: Title: The Melody of Metal Hearts

Once upon a time, in the futuristic city of N...

Gemini Flash: 761 characters generated
Preview: Unit 734, designated "Custodian," hummed softly, its optical sensors sweeping th...



### 🎯 Configuration Benefits

- **Environment integration** - Automatic loading from .env files
- **Validation** - Catch configuration errors early
- **Flexibility** - Multiple configuration methods
- **Security** - No hardcoded API keys in code
- **Easy switching** - Change models and parameters instantly
- **Cross-provider consistency** - Same patterns work everywhere
- **A/B testing ready** - Compare providers effortlessly

## 4. Error Handling & Resilience

### ❌ WITHOUT MERIT: Manual Error Handling

You'd need to implement all error handling manually:

In [57]:
import requests
import time
import random
import os

def robust_api_call_without_merit(prompt, max_retries=3):
    """Manual implementation of retries and error handling"""
    
    for attempt in range(max_retries):
        try:
            response = requests.post(
                'https://api.openai.com/v1/chat/completions',
                headers={'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}'},
                json={
                    'model': 'gpt-3.5-turbo',
                    'messages': [{'role': 'user', 'content': prompt}]
                },
                timeout=30
            )
            
            # Handle different error types manually
            if response.status_code == 429:  # Rate limit
                wait_time = 2 ** attempt + random.uniform(0, 1)
                print(f'Rate limited, waiting {wait_time:.1f}s...')
                time.sleep(wait_time)
                continue
                
            elif response.status_code == 500:  # Server error
                print(f'Server error, attempt {attempt + 1}/{max_retries}')
                time.sleep(1)
                continue
                
            elif response.status_code == 401:  # Auth error
                raise ValueError('Invalid API key')
                
            response.raise_for_status()
            return response.json()['choices'][0]['message']['content']
            
        except requests.exceptions.Timeout:
            print(f'Timeout, attempt {attempt + 1}/{max_retries}')
            if attempt == max_retries - 1:
                raise
                
        except requests.exceptions.ConnectionError:
            print(f'Connection error, attempt {attempt + 1}/{max_retries}')
            time.sleep(2 ** attempt)
            if attempt == max_retries - 1:
                raise
    
    raise Exception('Max retries exceeded')

# Usage - you need to handle errors yourself
print('=== Manual Error Handling Example ===')
try:
    result = robust_api_call_without_merit('What is AI?')
    print(f'Success: {result[:50]}...')
except Exception as e:
    print(f'Failed after retries: {e}')

# Problems with manual approach:
print('\n❌ Problems with manual error handling:')
print('- 50+ lines of complex error handling code')
print('- Easy to miss edge cases')
print('- No adaptive rate limiting')
print('- Inconsistent across different API calls')
print('- Hard to maintain and debug')

=== Manual Error Handling Example ===
Success: AI, or artificial intelligence, refers to the simu...

❌ Problems with manual error handling:
- 50+ lines of complex error handling code
- Easy to miss edge cases
- No adaptive rate limiting
- Inconsistent across different API calls
- Hard to maintain and debug


### ✅ WITH MERIT: Automatic Error Handling

MERIT handles all of this automatically:

In [58]:
from merit.api import OpenAIClient
from merit.api.run_config import with_retry, adaptive_throttle, with_adaptive_retry

# Basic usage - automatic error handling included (graceful mode)
print('=== Basic MERIT Error Handling (Graceful Mode) ===')
client = OpenAIClient(
    api_key='invalid',
    strict=False  # Graceful mode (default)
)

# This single line includes retries, rate limiting, validation
result = client.generate_text('What is AI?')
print(f'✅ Graceful mode success: {result[:50]}...')

# Strict mode - raises exceptions on failures
print('\n=== Strict Mode Error Handling ===')
strict_client = OpenAIClient(
    api_key='invalid',
    strict=True  # Strict mode - raises exceptions
)

try:
    result = strict_client.generate_text('What is AI?')
    print(f'✅ Strict mode success: {result[:50]}...')
except Exception as e:
    print(f'❌ Strict mode exception: {type(e).__name__}: {e}')

# Method-level override
print('\n=== Method-Level Strict Override ===')
graceful_client = OpenAIClient(
    api_key='invalid',
    strict=False  # Default graceful
)

# Override to strict for this specific call
try:
    result = graceful_client.generate_text('Hello', strict=True)
    print(f'✅ Method override success: {result}')
except Exception as e:
    print(f'❌ Method override exception: {type(e).__name__}')

# Graceful call (uses client default)
result = graceful_client.generate_text('Hello')
print(f'✅ Graceful call: {result}')

=== Basic MERIT Error Handling (Graceful Mode) ===


API call failed (graceful mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


✅ Graceful mode success: ...

=== Strict Mode Error Handling ===


API call failed (strict mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


❌ Strict mode exception: MeritAPIAuthenticationError: Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.

=== Method-Level Strict Override ===


API call failed (strict mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


❌ Method override exception: MeritAPIAuthenticationError


API call failed (graceful mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


✅ Graceful call: 


### 🛡️ Error Handling in Production

Real-world error scenarios and how MERIT handles them:

In [59]:
# Scenario 1: Invalid API key with strict mode
print('=== Scenario 1: Invalid API Key (Strict Mode) ===')
try:
    invalid_client = OpenAIClient(
        api_key='invalid-key-12345',
        strict=True  # Strict mode - will raise exceptions
    )
    result = invalid_client.generate_text('Test')
    print(f'❌ Unexpected success: {result}')
except Exception as e:
    print(f'✅ MERIT properly handled invalid API key: {type(e).__name__}')

# Scenario 1b: Invalid API key with graceful mode
print('\n=== Scenario 1b: Invalid API Key (Graceful Mode) ===')
invalid_graceful_client = OpenAIClient(
    api_key='invalid-key-12345',
    strict=False  # Graceful mode - returns None
)
result = invalid_graceful_client.generate_text('Test')
if result is None:
    print('✅ MERIT gracefully handled invalid API key: returned None/empty')
else:
    print(f'❌ Unexpected success: {result}')

# Scenario 2: Invalid model with strict mode
print('\n=== Scenario 2: Invalid Model (Strict Mode) ===')
try:
    invalid_model_client = OpenAIClient(
        api_key=OPENAI_API_KEY,
        model='invalid-model-12345',
        strict=True  # Strict mode - will raise exceptions
    )
    result = invalid_model_client.generate_text('Test')
    print(f'❌ Unexpected success: {result}')
except Exception as e:
    print(f'✅ MERIT properly handled invalid model: {type(e).__name__}')

# Scenario 3: Graceful degradation with proper error checking
print('\n=== Scenario 3: Graceful Degradation ===')

def resilient_app(prompts):
    """App that continues working even when some calls fail"""
    # Use graceful mode for resilient processing
    valid_client = OpenAIClient(
        api_key=OPENAI_API_KEY,
        strict=False  # Graceful mode
    )
    
    # Also test with invalid client for demonstration
    invalid_client = OpenAIClient(
        api_key='invalid-key-12345',
        strict=False  # Graceful mode
    )
    
    results = []
    
    for prompt in prompts:
        # Try with valid client
        result = valid_client.generate_text(prompt)
        if result and len(result.strip()) > 0:
            results.append({'prompt': prompt, 'result': result, 'status': 'success'})
        else:
            # Try with invalid client to show graceful handling
            result = invalid_client.generate_text(prompt)
            if result and len(result.strip()) > 0:
                results.append({'prompt': prompt, 'result': result, 'status': 'success'})
            else:
                results.append({'prompt': prompt, 'error': 'API call failed gracefully', 'status': 'failed'})
    
    return results

# Test with mix of valid prompts
test_prompts = [
    'What is Python?',
    'Explain machine learning',
    'What is the future of AI?'
]

results = resilient_app(test_prompts)

for i, result in enumerate(results):
    if result['status'] == 'success':
        print(f'✅ Prompt {i+1}: {result["result"][:50]}...')
    else:
        print(f'❌ Prompt {i+1}: {result["error"]}')

success_rate = len([r for r in results if r['status'] == 'success']) / len(results) * 100
print(f'\n📊 Success rate: {success_rate:.1f}%')

# Scenario 4: Demonstrating the difference
print('\n=== Scenario 4: Strict vs Graceful Comparison ===')

# Strict mode example
print('🔴 Strict Mode (Development/Debugging):')
strict_client = OpenAIClient(api_key='invalid-key', strict=True)
try:
    result = strict_client.generate_text('Test')
    print('❌ Should have failed')
except Exception as e:
    print(f'✅ Exception raised: {type(e).__name__} - Good for debugging!')

# Graceful mode example
print('🟢 Graceful Mode (Production/Evaluation):')
graceful_client = OpenAIClient(api_key='invalid-key', strict=False)
result = graceful_client.generate_text('Test')
if result is None or result == '':
    print('✅ Returned None/empty - Good for continued processing!')
else:
    print(f'❌ Unexpected result: {result}')


=== Scenario 1: Invalid API Key (Strict Mode) ===


API call failed (strict mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


✅ MERIT properly handled invalid API key: MeritAPIAuthenticationError

=== Scenario 1b: Invalid API Key (Graceful Mode) ===


API call failed (graceful mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


❌ Unexpected success: 

=== Scenario 2: Invalid Model (Strict Mode) ===


...

✅ MERIT properly handled invalid model: MeritAPIResourceNotFoundError

=== Scenario 3: Graceful Degradation ===
✅ Prompt 1: Python is a high-level, interpreted programming la...
✅ Prompt 2: Machine learning is a subset of artificial intelli...
✅ Prompt 3: The future of AI is likely to involve continued ad...

📊 Success rate: 100.0%

=== Scenario 4: Strict vs Graceful Comparison ===
🔴 Strict Mode (Development/Debugging):


API call failed (strict mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


✅ Exception raised: MeritAPIAuthenticationError - Good for debugging!
🟢 Graceful Mode (Production/Evaluation):


API call failed (graceful mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


✅ Returned None/empty - Good for continued processing!


### 🎯 Error Handling Benefits

- **Automatic retries** - Handles transient failures without code changes
- **Rate limiting protection** - Prevents 429 errors with adaptive delays
- **Adaptive throttling** - Learns optimal request timing automatically
- **Graceful degradation** - Applications continue working when possible
- **Consistent behavior** - Same error handling across all API calls
- **Production ready** - Battle-tested error handling patterns
- **Zero configuration** - Works out of the box with sensible defaults

## 5. Caching & Performance

### ❌ WITHOUT MERIT: No Caching

In [60]:
import time
import requests
import os

def get_embedding_without_cache(text):
    """Every call hits the API - expensive and slow"""
    api_key = os.getenv('OPENAI_API_KEY')
    response = requests.post(
        'https://api.openai.com/v1/embeddings',
        headers={'Authorization': f'Bearer {api_key}'},
        json={'input': text, 'model': 'text-embedding-ada-002'}
    )
    response.raise_for_status()
    return response.json()['data'][0]['embedding']

# Simulate repeated calls with same text
text = "What is machine learning?"

print("❌ WITHOUT CACHING:")
start_time = time.time()

# Each call takes ~500ms and costs money
embeddings_no_cache = []
for i in range(3):
    call_start = time.time()
    embedding = get_embedding_without_cache(text)
    call_time = time.time() - call_start
    embeddings_no_cache.append(embedding)
    print(f"Call {i+1}: {call_time:.2f}s - Got {len(embedding)} dimensions")

total_time = time.time() - start_time
print(f"Total time: {total_time:.2f}s")
print(f"Cost: 3 API calls = 3x the cost")
print(f"Average call time: {total_time/3:.2f}s")


❌ WITHOUT CACHING:
Call 1: 0.97s - Got 1536 dimensions
Call 2: 0.97s - Got 1536 dimensions
Call 3: 1.14s - Got 1536 dimensions
Total time: 3.08s
Cost: 3 API calls = 3x the cost
Average call time: 1.03s


### ✅ WITH MERIT: Automatic Caching

In [61]:
from merit.api import OpenAIClient
import time

client = OpenAIClient(api_key=OPENAI_API_KEY)
text = "What is machine learning?"

print("✅ WITH MERIT CACHING:")
start_time = time.time()

# First call hits the API
call_start = time.time()
embedding1 = client.get_embeddings([text])
first_call_time = time.time() - call_start
print(f"Call 1 (API): {first_call_time:.2f}s - Got {len(embedding1[0])} dimensions")

# Subsequent calls use cache
for i in range(2, 4):
    call_start = time.time()
    embedding = client.get_embeddings([text])
    call_time = time.time() - call_start
    print(f"Call {i} (cached): {call_time:.4f}s - Got {len(embedding[0])} dimensions")

total_time = time.time() - start_time
print(f"\nTotal time: {total_time:.2f}s")
print(f"Cost: 1 API call instead of 3 = 67% cost savings")
print(f"Speed: ~{first_call_time/0.001:.0f}x faster for cached calls")

# Demonstrate cache persistence across different texts
print("\n=== Cache Efficiency with Multiple Texts ===")
texts = [
    "What is machine learning?",  # Already cached
    "What is deep learning?",     # New - will hit API
    "What is machine learning?",  # Cached again
    "What is artificial intelligence?",  # New - will hit API
    "What is deep learning?",     # Now cached
]

api_calls = 0
cache_hits = 0
total_start = time.time()

for i, text in enumerate(texts):
    call_start = time.time()
    embedding = client.get_embeddings([text])
    call_time = time.time() - call_start
    
    if call_time > 0.1:  # Likely an API call
        api_calls += 1
        print(f"Text {i+1}: {call_time:.2f}s (API) - {text[:30]}...")
    else:  # Likely cached
        cache_hits += 1
        print(f"Text {i+1}: {call_time:.4f}s (cached) - {text[:30]}...")

total_time = time.time() - total_start
print(f"\nSummary:")
print(f"Total time: {total_time:.2f}s")
print(f"API calls: {api_calls}")
print(f"Cache hits: {cache_hits}")
print(f"Cache hit rate: {cache_hits/(api_calls+cache_hits)*100:.1f}%")
print(f"Cost savings: {cache_hits/(api_calls+cache_hits)*100:.1f}%")


✅ WITH MERIT CACHING:
Call 1 (API): 0.00s - Got 1536 dimensions
Call 2 (cached): 0.0000s - Got 1536 dimensions
Call 3 (cached): 0.0000s - Got 1536 dimensions

Total time: 0.00s
Cost: 1 API call instead of 3 = 67% cost savings
Speed: ~0x faster for cached calls

=== Cache Efficiency with Multiple Texts ===
Text 1: 0.0000s (cached) - What is machine learning?...
Text 2: 0.0000s (cached) - What is deep learning?...
Text 3: 0.0000s (cached) - What is machine learning?...
Text 4: 0.0000s (cached) - What is artificial intelligenc...
Text 5: 0.0000s (cached) - What is deep learning?...

Summary:
Total time: 0.00s
API calls: 0
Cache hits: 5
Cache hit rate: 100.0%
Cost savings: 100.0%


## 6. Advanced Features: Retry & Throttling Decorators

MERIT provides advanced decorators for handling rate limiting and retries. Let's test these features:

### ❌ WITHOUT DECORATORS: Manual Retry Logic

First, let's see what manual retry logic looks like:


In [62]:
# Manual retry implementation - complex and error-prone
import time
import random
from merit.api import OpenAIClient

def manual_retry_api_call(prompt, max_retries=3):
    """Manual implementation of retries - lots of boilerplate"""
    
    client = OpenAIClient(api_key='invalid-key', strict=True)  # Use invalid key to trigger errors
    
    for attempt in range(max_retries):
        try:
            result = client.generate_text(prompt)
            return result
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Failed after {max_retries} attempts: {e}")
                raise
            
            # Manual backoff calculation
            wait_time = (2 ** attempt) + random.uniform(0, 1)
            print(f"Attempt {attempt + 1} failed, waiting {wait_time:.1f}s...")
            time.sleep(wait_time)
    
    return None

# Test manual retry
print("=== Manual Retry Implementation ===")
try:
    result = manual_retry_api_call("Test prompt")
    print(f"Success: {result}")
except Exception as e:
    print(f"Final failure: {type(e).__name__}")


=== Manual Retry Implementation ===


API call failed (strict mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


Attempt 1 failed, waiting 1.6s...


API call failed (strict mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


Attempt 2 failed, waiting 2.2s...


API call failed (strict mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


Failed after 3 attempts: Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.
Final failure: MeritAPIAuthenticationError


### ✅ WITH MERIT DECORATORS: Automatic Retry & Throttling

Now let's use MERIT's decorators:


In [63]:
# Import the decorators
from merit.api.run_config import adaptive_throttle, with_retry, with_adaptive_retry
from merit.api import OpenAIClient
import time

# Test 1: Basic retry decorator
print("=== Test 1: Basic Retry Decorator ===")

@with_retry(max_retries=3, backoff_factor=0.5)
def test_basic_retry():
    """Function with automatic retry on failures"""
    client = OpenAIClient(api_key='invalid-key', strict=True)
    return client.generate_text("Test prompt")

try:
    result = test_basic_retry()
    print(f"✅ Success: {result}")
except Exception as e:
    print(f"❌ Final failure after retries: {type(e).__name__}")

# Test 2: Adaptive throttling decorator
print("\n=== Test 2: Adaptive Throttling ===")

@adaptive_throttle
def test_throttling(call_number):
    """Function with adaptive rate limiting"""
    print(f"Making API call #{call_number}")
    # Simulate API call timing
    start_time = time.time()
    time.sleep(0.1)  # Simulate API response time
    duration = time.time() - start_time
    print(f"Call #{call_number} completed in {duration:.3f}s")
    return f"Response {call_number}"

# Make several calls to see adaptive throttling in action
print("Making 5 throttled calls...")
for i in range(1, 6):
    result = test_throttling(i)
    print(f"Got: {result}")

# Test 3: Combined adaptive retry decorator
print("\n=== Test 3: Combined Adaptive Retry ===")

@with_adaptive_retry(max_retries=2)
def test_combined_features(success_rate=0.3):
    """Function that randomly fails to test retry + throttling"""
    import random
    
    if random.random() < success_rate:
        return "Success!"
    else:
        # Simulate different types of failures
        failure_type = random.choice(['rate_limit', 'connection', 'server'])
        if failure_type == 'rate_limit':
            from merit.api.errors import MeritAPIRateLimitError
            raise MeritAPIRateLimitError("Simulated rate limit")
        elif failure_type == 'connection':
            from merit.api.errors import MeritAPIConnectionError
            raise MeritAPIConnectionError("Simulated connection error")
        else:
            from merit.api.errors import MeritAPIServerError
            raise MeritAPIServerError("Simulated server error")

# Test the combined decorator
print("Testing combined retry + throttling (may take a moment)...")
for i in range(3):
    try:
        result = test_combined_features(success_rate=0.7)  # 70% success rate
        print(f"✅ Attempt {i+1}: {result}")
    except Exception as e:
        print(f"❌ Attempt {i+1} failed: {type(e).__name__}")


=== Test 1: Basic Retry Decorator ===


API call failed (strict mode): Authentication failed (Error Code: MAPI-001) 

Troubleshooting: Please check your API key and ensure it is valid. Verify that the API key has been correctly set in your configuration. If using environment variables, ensure MERIT_API_KEY or the service-specific API key variable (e.g., OPENAI_API_KEY) is correctly set.


❌ Final failure after retries: MeritAPIAuthenticationError

=== Test 2: Adaptive Throttling ===
Making 5 throttled calls...
Making API call #1
Call #1 completed in 0.105s
Got: Response 1
Making API call #2
Call #2 completed in 0.105s
Got: Response 2
Making API call #3
Call #3 completed in 0.101s
Got: Response 3
Making API call #4
Call #4 completed in 0.106s
Got: Response 4
Making API call #5


API call failed with MeritAPIServerError, retrying in 0.62 seconds (retry 1/2)


Call #5 completed in 0.105s
Got: Response 5

=== Test 3: Combined Adaptive Retry ===
Testing combined retry + throttling (may take a moment)...
✅ Attempt 1: Success!
✅ Attempt 2: Success!
✅ Attempt 3: Success!


### 🔧 Real-World Usage: Decorating API Wrapper Functions

Here's how you'd use these decorators in practice:


In [64]:
# Real-world example: Robust API wrapper functions
from merit.api import OpenAIClient, GeminiClient
from merit.api.run_config import with_adaptive_retry

class RobustAPIService:
    """Example service using MERIT decorators for reliability"""
    
    def __init__(self):
        self.openai_client = OpenAIClient(strict=False)  # Graceful mode
        self.gemini_client = GeminiClient(strict=False)
    
    @with_adaptive_retry(max_retries=3)
    def generate_with_openai(self, prompt):
        """OpenAI generation with automatic retry and throttling"""
        return self.openai_client.generate_text(prompt)
    
    @with_adaptive_retry(max_retries=3)
    def generate_with_gemini(self, prompt):
        """Gemini generation with automatic retry and throttling"""
        return self.gemini_client.generate_text(prompt)
    
    def robust_generation(self, prompt):
        """Try multiple providers with automatic fallback"""
        # Try OpenAI first
        result = self.generate_with_openai(prompt)
        if result and len(result.strip()) > 0:
            return {"provider": "openai", "result": result}
        
        # Fallback to Gemini
        result = self.generate_with_gemini(prompt)
        if result and len(result.strip()) > 0:
            return {"provider": "gemini", "result": result}
        
        return {"provider": "none", "result": "All providers failed"}

# Test the robust service
print("\n=== Real-World Usage Example ===")
service = RobustAPIService()

# Test with a simple prompt
test_prompt = "What is artificial intelligence?"
result = service.robust_generation(test_prompt)
print(f"Provider: {result['provider']}")
print(f"Result: {result['result'][:100]}..." if result['result'] else "No result")



=== Real-World Usage Example ===
Provider: openai
Result: Artificial intelligence (AI) is the simulation of human intelligence processes by machines, especial...


In [65]:
# MERIT Decorators Demo: Retries + Throttling in Action
import time
import random
from merit.api.run_config import with_adaptive_retry, adaptive_throttle, with_retry
from merit.api.errors import MeritAPIRateLimitError, MeritAPIConnectionError

print("🔬 MERIT DECORATORS DEMONSTRATION")
print("=" * 50)

# Demo 1: Retry Decorator - Shows actual retry attempts
print("\n1️⃣ RETRY DECORATOR - Watch the retries happen:")
print("-" * 45)

@with_retry(max_retries=3, backoff_factor=0.3)
def flaky_api_call(call_id):
    """Simulates an API that fails 70% of the time"""
    print(f"   🔄 Attempting API call #{call_id}...")
    
    if random.random() < 0.7:  # 70% failure rate
        error_type = random.choice(['rate_limit', 'connection'])
        if error_type == 'rate_limit':
            print(f"   ❌ Rate limit hit for call #{call_id}")
            raise MeritAPIRateLimitError("API rate limit exceeded")
        else:
            print(f"   ❌ Connection failed for call #{call_id}")
            raise MeritAPIConnectionError("Connection timeout")
    else:
        print(f"   ✅ Success for call #{call_id}!")
        return f"Response from call #{call_id}"

# Test the retry decorator
for i in range(1, 4):
    try:
        result = flaky_api_call(i)
        print(f"   🎉 Final result: {result}\n")
    except Exception as e:
        print(f"   💥 Final failure: {type(e).__name__}\n")

# Demo 2: Adaptive Throttling - Shows delay adjustments
print("2️⃣ ADAPTIVE THROTTLING - Watch delays adjust:")
print("-" * 45)

call_count = 0

@adaptive_throttle
def throttled_api_call():
    """Shows adaptive throttling in action"""
    global call_count
    call_count += 1
    
    start_time = time.time()
    
    # Simulate API processing time
    time.sleep(0.05)  # 50ms API response time
    
    duration = time.time() - start_time
    print(f"   📡 Call #{call_count} completed in {duration:.3f}s")
    
    # Simulate occasional rate limiting to show adaptation
    if call_count == 3:
        print(f"   ⚠️  Simulating rate limit on call #{call_count}")
        raise MeritAPIRateLimitError("Rate limit - delay will increase")
    
    return f"Success #{call_count}"

# Make several calls to see throttling adapt
print("Making 6 throttled calls (watch the delays):")
for i in range(6):
    try:
        start = time.time()
        result = throttled_api_call()
        total_time = time.time() - start
        print(f"   ✅ {result} (total time: {total_time:.3f}s)")
    except Exception as e:
        print(f"   ❌ {type(e).__name__} - delay will increase")

# Demo 3: Combined Adaptive Retry - Shows both features together
print("\n3️⃣ COMBINED ADAPTIVE RETRY - Retries + Throttling:")
print("-" * 50)

attempt_count = 0

@with_adaptive_retry(max_retries=2)
def robust_api_call(task_name):
    """Shows combined retry + throttling"""
    global attempt_count
    attempt_count += 1
    
    print(f"   🚀 Processing '{task_name}' (attempt #{attempt_count})")
    
    # Simulate different failure scenarios
    failure_chance = random.random()
    
    if failure_chance < 0.4:  # 40% chance of rate limit
        print(f"   ⏳ Rate limited - will retry with increased delay")
        raise MeritAPIRateLimitError("Rate limit exceeded")
    elif failure_chance < 0.6:  # 20% chance of connection error
        print(f"   🔌 Connection error - will retry")
        raise MeritAPIConnectionError("Connection failed")
    else:  # 40% chance of success
        print(f"   🎯 Successfully processed '{task_name}'")
        return f"Completed: {task_name}"

# Test combined features
tasks = ["Data Analysis", "Report Generation", "Model Training"]

for task in tasks:
    attempt_count = 0  # Reset for each task
    try:
        result = robust_api_call(task)
        print(f"   ✅ {result}\n")
    except Exception as e:
        print(f"   💥 Task '{task}' failed after retries: {type(e).__name__}\n")

print("🎯 SUMMARY:")
print("✅ Retry decorator: Automatically retries failed calls with exponential backoff")
print("✅ Adaptive throttling: Learns optimal timing and adjusts delays dynamically") 
print("✅ Combined approach: Production-ready resilience for unreliable APIs")
print("\n💡 These decorators can be applied to any function that makes API calls!")


API call failed with MeritAPIRateLimitError, retrying in 0.32 seconds (retry 1/3)


🔬 MERIT DECORATORS DEMONSTRATION

1️⃣ RETRY DECORATOR - Watch the retries happen:
---------------------------------------------
   🔄 Attempting API call #1...
   ✅ Success for call #1!
   🎉 Final result: Response from call #1

   🔄 Attempting API call #2...
   ❌ Rate limit hit for call #2


API call failed with MeritAPIRateLimitError, retrying in 0.73 seconds (retry 2/3)


   🔄 Attempting API call #2...
   ❌ Rate limit hit for call #2


API call failed with MeritAPIRateLimitError, retrying in 1.20 seconds (retry 3/3)


...

   🚀 Processing 'Data Analysis' (attempt #3)
   ⏳ Rate limited - will retry with increased delay
   💥 Task 'Data Analysis' failed after retries: MeritAPIRateLimitError

   🚀 Processing 'Report Generation' (attempt #1)
   🎯 Successfully processed 'Report Generation'
   ✅ Completed: Report Generation



API call failed with MeritAPIConnectionError, retrying in 0.58 seconds (retry 1/2)


   🚀 Processing 'Model Training' (attempt #1)
   🔌 Connection error - will retry


API call failed with MeritAPIRateLimitError, retrying in 1.24 seconds (retry 2/2)


   🚀 Processing 'Model Training' (attempt #2)
   ⏳ Rate limited - will retry with increased delay
   🚀 Processing 'Model Training' (attempt #3)
   🎯 Successfully processed 'Model Training'
   ✅ Completed: Model Training

🎯 SUMMARY:
✅ Retry decorator: Automatically retries failed calls with exponential backoff
✅ Adaptive throttling: Learns optimal timing and adjusts delays dynamically
✅ Combined approach: Production-ready resilience for unreliable APIs

💡 These decorators can be applied to any function that makes API calls!


### 🎯 Decorator Benefits Summary

 ✅ **AUTOMATIC RETRY LOGIC:**
- Handles transient failures automatically
- Exponential backoff with jitter
- Configurable retry attempts

 ✅ **ADAPTIVE RATE LIMITING:**
- Learns optimal request timing
- Prevents 429 rate limit errors
- Reduces delay after successful calls

 ✅ **PRODUCTION READY:**
- Thread-safe implementation
- Comprehensive error handling
- Detailed logging and statistics

 ✅ **EASY TO USE:**
- Simple decorator syntax
- Works with any function
- Configurable parameters

 🔥 **PERFECT FOR:**
- High-volume API usage
- Production applications
- Unreliable network conditions
- Cost-sensitive operations


## . Next Steps

Now that you've seen how MERIT simplifies LLM API usage, here are some next steps:

### 🔗 Learn More
- Check out the full MERIT documentation
- Explore other MERIT modules (evaluation, monitoring, etc.)
- Join the MERIT community

### 🚀 Build Something
- Create your own RAG system
- Build a multi-model chatbot
- Implement cost-effective batch processing

### 💡 Contribute
- Report issues or suggest features
- Contribute to the open-source project
- Share your MERIT success stories